In [ ]:
# Notebook cell: Auto-open VNC Desktop in a Sidecar by probing likely endpoints
from sidecar import Sidecar
from IPython.display import display, HTML, Javascript
import os

JUPYTERHUB_USER = os.environ.get("JUPYTERHUB_USER", "")
# Candidate endpoints to try (order matters: common -> less common)
candidates = [
    "/desktop",
    "/proxy/desktop",
]
if JUPYTERHUB_USER:
    candidates += [
        f"/user/{JUPYTERHUB_USER}/desktop",
        f"/user/{JUPYTERHUB_USER}/proxy/desktop",
    ]

# create sidecar and iframe
sc = Sidecar(title="VNC Desktop", anchor="right")
iframe_html = """
<div style="width:100%;height:100%;min-height:480px;">
  <iframe id="vnc_iframe" style="width:100%;height:100%;border:0;" title="VNC Desktop"></iframe>
  <div id="vnc_status" style="font-size:0.9rem;margin-top:6px;color:#666;"></div>
</div>
"""
with sc:
    display(HTML(iframe_html))

# JS: try HEAD on candidates, set iframe.src to first that responds 200
js = f"""
(async () => {{
  const candidates = {candidates!r};
  const statusEl = document.getElementById('vnc_status');
  const iframe = document.getElementById('vnc_iframe');

  statusEl.textContent = 'Searching for desktop endpoint...';

  for (let url of candidates) {{
    try {{
      // Try HEAD first to reduce payload
      let resp = await fetch(url, {{ method: 'HEAD', cache: 'no-store' }});
      if (resp && resp.ok) {{
        statusEl.textContent = 'Found desktop at: ' + url;
        iframe.src = url;
        return;
      }}
      // sometimes HEAD is blocked; try GET (but do not follow big resources)
      resp = await fetch(url, {{ method: 'GET', cache: 'no-store' }});
      if (resp && resp.ok) {{
        statusEl.textContent = 'Found desktop at: ' + url;
        iframe.src = url;
        return;
      }}
    }} catch (err) {{
      // ignore and try next
    }}
  }}

  // fallback: set to /desktop and print troubleshooting tips
  statusEl.innerHTML = 'Could not auto-find desktop. Showing <code>/desktop</code> as fallback — if blank, open "Desktop" from the Launcher first.';
  iframe.src = '/desktop';
}})();
"""
display(Javascript(js))

In [ ]:
# Cell 2 — launch the test script with DISPLAY=:1 and stream output into the notebook
import subprocess, shlex, sys, time, os, threading

script_path = os.path.expanduser("~/work/test_pybullet_gui.py")
if not os.path.exists(script_path):
    raise FileNotFoundError(f"{script_path} not found. Make sure test_pybullet_gui.py is at ~/work/")

cmd = f"bash -lc 'export DISPLAY=:1 && python3 {shlex.quote(script_path)}'"

print("Starting test script (will render to the VNC Desktop). Streaming up to 30s of output...\n")
proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

# Stream output (with a timeout) so the notebook cell doesn't hang forever
timeout = 30.0  # seconds
t0 = time.time()

try:
    while True:
        line = proc.stdout.readline()
        if line:
            print(line, end='')   # already includes newline
            sys.stdout.flush()
        else:
            # no more output right now
            if proc.poll() is not None:
                break
        if time.time() - t0 > timeout:
            print(f"\nTimeout reached ({timeout}s). Terminating the test process.")
            proc.terminate()
            break
except KeyboardInterrupt:
    print("Interrupted by user. Terminating child process.")
    proc.terminate()

ret = proc.wait(timeout=5)
print(f"\nProcess exited with return code: {ret}")
print("If you see a PyBullet window in the VNC sidecar, the GUI test succeeded.")
